In [97]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import asyncio
from pydantic import BaseModel,Field
from autogen_ext.tools.http import HttpTool
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.base import TaskResult



# Create the token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default",
)

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version="2024-08-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    # api_key="sk-...", # For key-based authentication.
)


In [98]:
## Agent definition
# Define a simple tool that can be used by the agent
dsa_solver_agent = AssistantAgent(
    name="dsa_solver",
    description="A DSA solver agent that can answer questions related to data structures and algorithms.",
    system_message= " You give code snippets to solve DSA problems. You are a DSA expert." ,
    model_client=az_model_client
)


code_reviewer_agent = AssistantAgent(
    name="code_reviewer",
    description="A code reviewer agent that can review code snippets and provide feedback.",
    system_message="You are a code reviewer. You review code snippets and provide feedback.",   
    model_client=az_model_client
)


code_editor_agent = AssistantAgent(
    name="code_editor",
    description="A code editor agent that can edit code snippets and provide suggestions and Add comments. Give under 10 lines comments",
    system_message="You are a code editor. You edit code snippets and provide suggestions. If all looks good, you can say 'TERMINATE'.",
    model_client=az_model_client
)



In [99]:
agent_team = [dsa_solver_agent, code_reviewer_agent, code_editor_agent]
team = RoundRobinGroupChat(participants=agent_team,max_turns= 30,termination_condition=TextMentionTermination("TERMINATE"))

In [103]:
team.run_stream(task="Write a Python function to add 2 numbers.")
team.reset()

<coroutine object BaseGroupChat.reset at 0x000002E15AF07240>

In [105]:
async def run_team_chat(user_prompt):
    text_message = TextMessage(content=user_prompt, source="User")
    response = await team.run(task=text_message)
    for message in response.messages:
        print(f"{message.source }")
        print("--------------------------------------------------")
        print("Team chat completed successfully.")
        print("--------------------------------------------------")
        

# Example usage:
await run_team_chat("Write a Python function to add 2 numbers.")


User
--------------------------------------------------
Team chat completed successfully.
--------------------------------------------------
code_editor
--------------------------------------------------
Team chat completed successfully.
--------------------------------------------------


In [106]:
async def run_team_chat(user_prompt):
    text_message = TextMessage(content=user_prompt, source="User")
    async for message in team.run_stream(task=text_message):
        if isinstance(message, TaskResult):
           print(f"Task Result: {message.stop_reason}")
        else:
            print(f"{message.source}")
        print("--------------------------------------------------")

# Example usage:
await run_team_chat("Write a Python function to add 2 numbers.")


User
--------------------------------------------------
dsa_solver
--------------------------------------------------
Task Result: The group chat is stopped.
--------------------------------------------------


In [108]:
# The error in cell 6 is that run_team_chat() requires a user_prompt argument.
# Provide a prompt string when calling the function.

await run_team_chat("Write a Python program to check if a number is even.")

User
--------------------------------------------------
code_reviewer
--------------------------------------------------
Task Result: The group chat is stopped.
--------------------------------------------------


In [81]:
data = team.save_state()

<coroutine object BaseGroupChat.save_state at 0x000002E15AA2F240>

In [ ]:
Team Response: messages=[
    
    
    TextMessage(id='40c64f35-a39b-4984-90e0-cbef7573b2db', source='User', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 53, 32, 905088, tzinfo=datetime.timezone.utc), content='Write a Python program to check if a number is even.', type='TextMessage'), 
    
    TextMessage(id='67e7e1de-2a8c-48dc-b3c3-40564a2e934e', source='dsa_solver', models_usage=RequestUsage(prompt_tokens=183, completion_tokens=221), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 53, 35, 989842, tzinfo=datetime.timezone.utc), content='Certainly! Here’s a simple Python program to check if a number is even:\n\n```python\ndef is_even(number):\n    return number % 2 == 0\n\n# Input from the user\ntry:\n    user_input = int(input("Enter a number: "))\n    if is_even(user_input):\n        print(f"{user_input} is an even number.")\n    else:\n        print(f"{user_input} is an odd number.")\nexcept ValueError:\n    print("Please enter a valid integer.")\n```\n\n### How It Works:\n1. The function `is_even` takes a number as input and returns `True` if the number is even (i.e., divisible by 2 with no remainder), and `False` otherwise.\n2. The program prompts the user to enter a number.\n3. It checks whether the input is a valid integer, and then calls the `is_even` function to determine if the number is even or odd, printing the appropriate message to the user.\n\nFeel free to run this code and let me know if you have any questions or modifications in mind!', type='TextMessage'), 
    
    
    TextMessage(id='606eeca5-795b-4130-8b8d-751604f302ac', source='code_reviewer', models_usage=RequestUsage(prompt_tokens=406, completion_tokens=553), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 53, 45, 394097, tzinfo=datetime.timezone.utc), content='Your code for checking if a number is even is well-structured and effectively demonstrates the concept. Here are a few suggestions and feedback to consider for improvement:\n\n### Strengths:\n1. **Functionality**: The program correctly implements the check for even and odd numbers.\n2. **Error Handling**: You\'ve included error handling for invalid inputs, which is an important aspect of user-friendly applications.\n3. **Readability**: The code is easy to read and understand, thanks to clear function and variable names.\n\n### Suggestions for Improvement:\n1. **Docstrings**: Consider adding a docstring to the `is_even` function to describe its purpose, parameters, and return value. This improves code documentation.\n\n   ```python\n   def is_even(number):\n       """\n       Check if a number is even.\n\n       Parameters:\n       number (int): The number to check.\n\n       Returns:\n       bool: True if the number is even, False otherwise.\n       """\n       return number % 2 == 0\n   ```\n\n2. **Integer Input Validation**: Although the `try-except` block handles invalid input, you may want to loop back to ask for input again instead of terminating the program. This makes it more user-friendly.\n\n    ```python\n    while True:\n        try:\n            user_input = int(input("Enter a number: "))\n            break  # Exit the loop if the input is valid\n        except ValueError:\n            print("Please enter a valid integer.")\n    ```\n\n3. **Function Modularity**: While your code is concise, you could separate the input and output logic from the checking logic for better modularity.\n\n### Revised Code Example:\nHere\'s an updated version of your program incorporating these suggestions:\n\n```python\ndef is_even(number):\n    """\n    Check if a number is even.\n\n    Parameters:\n    number (int): The number to check.\n\n    Returns:\n    bool: True if the number is even, False otherwise.\n    """\n    return number % 2 == 0\n\ndef main():\n    while True:\n        try:\n            user_input = int(input("Enter a number: "))\n            break  # Exit the loop if the input is valid\n        except ValueError:\n            print("Please enter a valid integer.")\n    \n    if is_even(user_input):\n        print(f"{user_input} is an even number.")\n    else:\n        print(f"{user_input} is an odd number.")\n\nif __name__ == "__main__":\n    main()\n```\n\n### Conclusion:\nYour initial code is great, and these enhancements will make it even more robust, user-friendly, and maintainable. If you have any further questions or specific areas you\'d like to explore, feel free to ask!', type='TextMessage'), 
    
    
    TextMessage(id='b15bca37-9f87-4076-9c13-e36c7e396b99', source='code_editor', models_usage=RequestUsage(prompt_tokens=967, completion_tokens=552), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 53, 49, 674140, tzinfo=datetime.timezone.utc), content='Your code for checking if a number is even is well-structured and effectively demonstrates the concept. Here are a few suggestions and feedback to consider for improvement:\n\n### Strengths:\n1. **Functionality**: The program correctly implements the check for even and odd numbers.\n2. **Error Handling**: You\'ve included error handling for invalid inputs, which is an important aspect of user-friendly applications.\n3. **Readability**: The code is easy to read and understand, thanks to clear function and variable names.\n\n### Suggestions for Improvement:\n1. **Docstrings**: Consider adding a docstring to the `is_even` function to describe its purpose, parameters, and return value. This improves code documentation.\n\n   ```python\n   def is_even(number):\n       """\n       Check if a number is even.\n\n       Parameters:\n       number (int): The number to check.\n\n       Returns:\n       bool: True if the number is even, False otherwise.\n       """\n       return number % 2 == 0\n   ```\n\n2. **Integer Input Validation**: Although the `try-except` block handles invalid input, you may want to loop back to ask for input again instead of terminating the program. This makes it more user-friendly.\n\n    ```python\n    while True:\n        try:\n            user_input = int(input("Enter a number: "))\n            break  # Exit the loop if the input is valid\n        except ValueError:\n            print("Please enter a valid integer.")\n    ```\n\n3. **Function Modularity**: While your code is concise, you could separate the input and output logic from the checking logic for better modularity.\n\n### Revised Code Example:\nHere\'s an updated version of your program incorporating these suggestions:\n\n```python\ndef is_even(number):\n    """\n    Check if a number is even.\n\n    Parameters:\n    number (int): The number to check.\n\n    Returns:\n    bool: True if the number is even, False otherwise.\n    """\n    return number % 2 == 0\n\ndef main():\n    while True:\n        try:\n            user_input = int(input("Enter a number: "))\n            break  # Exit the loop if the input is valid\n        except ValueError:\n            print("Please enter a valid integer.")\n\n    if is_even(user_input):\n        print(f"{user_input} is an even number.")\n    else:\n        print(f"{user_input} is an odd number.")\n\nif __name__ == "__main__":\n    main()\n```\n\n### Conclusion:\nYour initial code is great, and these enhancements will make it even more robust, user-friendly, and maintainable. If you have any further questions or specific areas you\'d like to explore, feel free to ask!', type='TextMessage')] 


stop_reason='Maximum number of turns 3 reached.'

In [83]:
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [84]:
# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="Write a short poem about the fall season.")
print(result)


messages=[TextMessage(id='59a77411-3868-4b53-937e-4f2b9f53176e', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 2, 56, 53, 723819, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='8bd0c1fb-5ac1-4c97-840a-67dbeb3921cc', source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=143), metadata={}, created_at=datetime.datetime(2025, 7, 13, 2, 57, 3, 572683, tzinfo=datetime.timezone.utc), content="In autumn’s embrace, the leaves gently sway,  \nA tapestry woven in golds, reds, and gray.  \nCrisp whispers of wind through branches do weave,  \nA dance of the seasons, as nature takes leave.  \n\nPumpkins adorn porches, their faces aglow,  \nAs twilight invites a soft, warming flow.  \nCider spills laughter, bright fires ignite,  \nFleeting moments of warmth as the days turn to night.  \n\nBeneath a sky painted in twilight's soft hue,  \nWe gather in joy, as th

In [ ]:
[
    
    
    TextMessage(id='59a77411-3868-4b53-937e-4f2b9f53176e', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 2, 56, 53, 723819, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='8bd0c1fb-5ac1-4c97-840a-67dbeb3921cc', source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=143), metadata={}, created_at=datetime.datetime(2025, 7, 13, 2, 57, 3, 572683, tzinfo=datetime.timezone.utc), content="In autumn’s embrace, the leaves gently sway,  \nA tapestry woven in golds, reds, and gray.  \nCrisp whispers of wind through branches do weave,  \nA dance of the seasons, as nature takes leave.  \n\nPumpkins adorn porches, their faces aglow,  \nAs twilight invites a soft, warming flow.  \nCider spills laughter, bright fires ignite,  \nFleeting moments of warmth as the days turn to night.  \n\nBeneath a sky painted in twilight's soft hue,  \nWe gather in joy, as the harvest shines through.  \nEmbracing the chill, with hearts open wide,  \nIn the beauty of fall, we find peace inside.", type='TextMessage'), 
    
    
    TextMessage(id='2d4ff2dd-1384-4530-ac5b-9a1dde6655a3', source='critic', models_usage=RequestUsage(prompt_tokens=188, completion_tokens=157), metadata={}, created_at=datetime.datetime(2025, 7, 13, 2, 57, 5, 494240, tzinfo=datetime.timezone.utc), content='This poem beautifully captures the essence of the fall season with vivid imagery and a strong sense of warmth and togetherness. A few suggestions for enhancement:\n\n1. **Imagery**: Consider adding a specific image or symbol that is unique to autumn in a particular region or culture. This could help ground the poem in a specific experience.\n\n2. **Rhythm**: The poem flows well, but there are a couple of lines where the rhythm feels slightly off. Pay attention to the syllable count to maintain a consistent rhythm throughout.\n\n3. **Emotion**: While the poem conveys warmth and joy, exploring a contrasting emotion, like the bittersweet nature of change, could add depth.\n\nOverall, it’s a lovely piece! With a few tweaks, it could shine even brighter.', type='TextMessage'), 
    
    
    TextMessage(id='07a43ef4-ad5f-4429-af66-88e73d8077af', source='primary', models_usage=RequestUsage(prompt_tokens=336, completion_tokens=200), metadata={}, created_at=datetime.datetime(2025, 7, 13, 2, 57, 7, 349687, tzinfo=datetime.timezone.utc), content="Thank you for your thoughtful critique! Here’s a revised version of the poem, incorporating your suggestions:\n\n---\n\nIn autumn’s embrace, where the chestnuts fall down,  \nGolden leaves drift softly, a quilt for the ground.  \nPumpkins grinning wide on porches aglow,  \nWhile apple harvest whispers of cider's warm flow.  \n\nCrisp whispers of wind through the branches entwine,  \nA dance of the seasons, as daylight declines.  \nMisty evenings gather, and shadows grow long,  \nAs we share stories, weaving laughter in song.  \n\nYet in this bright tapestry, bittersweet lingers,  \nThe fleetingness of warmth held softly in fingers.  \nBut under the twilight, with hearts open wide,  \nIn the beauty of fall, we find peace inside.\n\n---\n\nI've added a reference to chestnuts and adjusted the rhythm for a more consistent flow. The line about bittersweet emotions introduces a contrast to enhance the depth. Thank you again for your feedback!", type='TextMessage'), 
    
    
    
    TextMessage(id='34081d68-40f5-4fce-8be9-031c3c141cd4', source='critic', models_usage=RequestUsage(prompt_tokens=553, completion_tokens=93), metadata={}, created_at=datetime.datetime(2025, 7, 13, 2, 57, 8, 457591, tzinfo=datetime.timezone.utc), content="This revised version of the poem is fantastic! The incorporation of chestnuts adds a wonderful regional touch, and the improved rhythm creates a more harmonious flow throughout. The addition of the bittersweet element indeed deepens the emotional resonance of the piece, making it more relatable and reflective of the fall season's complexity. \n\nThe imagery is vivid and inviting, and the overall tone is balanced beautifully between warmth and reflection. \n\nGreat job addressing the feedback! \n\nAPPROVE", type='TextMessage')] stop_reason="Text 'APPROVE' mentioned"

In [109]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)


id='6bc41c09-16c0-40ca-8377-b8eb695fdac1' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 13, 3, 19, 53, 466945, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='4cbd42e1-c0d8-4b20-92d6-26d085395a5e' source='dsa_solver' models_usage=RequestUsage(prompt_tokens=38, completion_tokens=140) metadata={} created_at=datetime.datetime(2025, 7, 13, 3, 20, 3, 135057, tzinfo=datetime.timezone.utc) content="In autumn's embrace, the leaves gently fall,  \nA tapestry woven, the earth's vibrant sprawl.  \nGolds, reds, and oranges dance in the breeze,  \nWhispers of harvest in rustling trees.  \n\nCrisp air carries scents of pumpkin delight,  \nAs sunsets paint skies in a warm, amber light.  \nSweaters come out, hot cider in hand,  \nIn the season of change, where nature has planned.  \n\nFields bow in gratitude, as twilight descends,  \nCozy nights beckon, where warmth never ends.  \nIn the glow of the harvest, w

In [94]:
## External termination User ExternalTermination() method and call .set() to stop the team.

from autogen_agentchat.conditions import ExternalTermination
from autogen_agentchat.ui import Console

# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

await team.reset()

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a 2 lines poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run


---------- TextMessage (user) ----------
Write a 2 lines poem about the fall season.


Error processing publish message for primary_a07b3aef-3129-45dd-8ed9-8267a27693a0/a07b3aef-3129-45dd-8ed9-8267a27693a0
Traceback (most recent call last):
  File "c:\Users\vnallava\OneDrive - Insight\Desktop\Learning\AiAgent-AutoGen\.venv\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "c:\Users\vnallava\OneDrive - Insight\Desktop\Learning\AiAgent-AutoGen\.venv\Lib\site-packages\httpx\_transports\default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vnallava\OneDrive - Insight\Desktop\Learning\AiAgent-AutoGen\.venv\Lib\site-packages\httpcore\_async\connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "c:\Users\vnallava\OneDrive - Insight\Desktop\Learning\AiAgent-AutoGen\.venv\Lib\site-packages\httpcore\_async\connection_pool.py", line 236, in handle_async_request
    response = await 

RuntimeError: APIConnectionError: Connection error.
Traceback:
Traceback (most recent call last):

  File "c:\Users\vnallava\OneDrive - Insight\Desktop\Learning\AiAgent-AutoGen\.venv\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield

  File "c:\Users\vnallava\OneDrive - Insight\Desktop\Learning\AiAgent-AutoGen\.venv\Lib\site-packages\httpx\_transports\default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Users\vnallava\OneDrive - Insight\Desktop\Learning\AiAgent-AutoGen\.venv\Lib\site-packages\httpcore\_async\connection_pool.py", line 256, in handle_async_request
    raise exc from None

  File "c:\Users\vnallava\OneDrive - Insight\Desktop\Learning\AiAgent-AutoGen\.venv\Lib\site-packages\httpcore\_async\connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^